# Legal Assistant Bot

This chatbot retrieves context from a proprietary datasource and the web to answer questions pertaining to federal laws in the United States of America (USA).  The proprietary datasource is a CSV file of all federal laws and their revision history in the USA.  The web data required to respond to the user's questions is retrieved using the You.com API.  The chatbot is implemented as an agent in Langchain.

## Install all required packages

In [48]:
! pip install langgraph==0.0.59
! pip install pandas==2.2.2
! pip install openai==1.30.3
! pip install langchain==0.2.1
! pip install langchain_community==0.2.1
! pip install langchain_openai==0.1.7
! pip install langchain_text_splitters==0.2.0
! pip install langchain_core==0.2.1
! pip install numpy==1.26.4

## Load in the US Laws dataset and create a Vector Database representation of this dataset, which will then be converted into a Langchain retriever and tool

In [49]:
# Let's take a look at our CSV dataset first
import pandas as pd

# The CSV file can be downloaded from: https://www.nature.com/articles/s41597-023-02758-z#Sec3
df = pd.read_csv("us_laws_dataset.csv")

df.head()

,row_number,action,Title,sal_volume,sal_page_start,BillCitation,congress_number,chapter,session_number,pl_no,date_of_passage,secondary_date,dates_conflict,Source,URL,alternate_sal_volume,alternate_sal_page_start,has_alternate_sal_citation
0,1,An Act,To regulate the time and manner of administeri...,1,23.0,NaN,1,1.0,1.0,NaN,1789-06-01,NaN,NaN,HeinOnline,NaN,NaN,NaN,False
1,2,An Act,"For laying a duty on goods, wares, and merchan...",1,24.0,NaN,1,2.0,1.0,NaN,1789-07-04,NaN,NaN,HeinOnline,NaN,NaN,NaN,False
2,3,An Act,Imposing duties on tonnage.,1,27.0,NaN,1,3.0,1.0,NaN,1789-07-20,NaN,NaN,HeinOnline,NaN,NaN,NaN,False
3,4,An Act,For establishing an executive department to be...,1,28.0,NaN,1,4.0,1.0,NaN,1789-07-27,NaN,NaN,HeinOnline,NaN,NaN,NaN,False
4,5,An Act,To regulate the collection of the duties impos...,1,29.0,NaN,1,5.0,1.0,NaN,1789-07-31,NaN,NaN,HeinOnline,NaN,NaN,NaN,False


In [50]:
import openai
import langchain
import os

In [51]:
os.environ["YDC_API_KEY"] = "<Insert your YDC API key here>"
os.environ["OPENAI_API_KEY"] = "<Insert your Open AI key here>"

In [52]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# The CSV file can be downloaded from: https://www.nature.com/articles/s41597-023-02758-z#Sec3
loader = CSVLoader(file_path = "us_laws_dataset.csv")

data = loader.load()

In [53]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split the document into chunks, and vectorize these chunks in a FAISS database
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
docs = text_splitter.split_documents(data)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(documents=docs, embedding=embeddings)

In [54]:
# test out the similarity search
query = "What laws and ammendments relate to perjury?"
response = db.similarity_search(query, k=10)
# let's look at the first 3 retrieved docs
response[:3]

[Document(page_content='row_number: 1885\naction: An Act\nTitle: In addition to the act, entitled "An act for the prompt settlement of public accounts," and for the punishment of the crime of perjury\nsal_volume: 3\nsal_page_start: 770\nBillCitation: NA\ncongress_number: 17\nchapter: 37\nsession_number: 2\npl_no: NA\ndate_of_passage: 1823-03-01\nsecondary_date: NA\ndates_conflict: NA\nSource: HeinOnline\nURL: NA\nalternate_sal_volume: NA\nalternate_sal_page_start: NA\nhas_alternate_sal_citation: FALSE', metadata={'source': 'us_laws_dataset.csv', 'row': 1884}),
 Document(page_content='row_number: 38724\naction: An Act\nTitle: An act to permit the use of unsworn declarations under penalty of perjury as evidence in Federal proceedings\nsal_volume: 90\nsal_page_start: 2534\nBillCitation: H.R. 15531\ncongress_number: 94\nchapter: NA\nsession_number: 2\npl_no: 94-550\ndate_of_passage: 1976-10-18\nsecondary_date: NA\ndates_conflict: FALSE\nSource: NA\nURL: https://www.govinfo.gov/content/pkg/

In [55]:
from langchain.tools.retriever import create_retriever_tool

# convert this retriver into a tool
db_retriever = db.as_retriever()
db_retriever_tool = create_retriever_tool(
    db_retriever,
    name = "law_dataset_retriever",
    description = "Retrieve relevant context from the US laws dataset."
)

## Instantiating the You.com Tool in Langchain

In [56]:
from langchain_community.tools.you import YouSearchTool
from langchain_community.utilities.you import YouSearchAPIWrapper

api_wrapper = YouSearchAPIWrapper(num_web_results = 10)
ydc_tool = YouSearchTool(api_wrapper=api_wrapper)

In [57]:
# test out the You.com search tool
response = ydc_tool.invoke("Tell me about a recent high-profile case related to antitrust in the USA?")
# let's look at the first 3 results
response[:3]

[Document(page_content='Meijer, Inc. v. Ferring B.V.; Ferring Pharmaceuticals, Inc.; and Aventis Pharmaceuticals [In Re: DDAVP Direct Purchaser Antitrust Litigation] U.S. v. Memphis Board of Realtors', metadata={'url': 'https://www.justice.gov/atr/antitrust-case-filings-alpha', 'thumbnail_url': None, 'title': 'Antitrust Division | Antitrust Case Filings | United States Department of Justice', 'description': 'An official website of the United States government · Official websites use .gov A .gov website belongs to an official government organization in the United States'}),
 Document(page_content="Dentsply International, Inc. v. Antitrust Division of the United States Department of Justice · U.S. v. Freddy Deoliveira · U.S. v. Wilhelm DerMinassian · U.S. v. Eric Descouraux · Leinani Deslandes, Stephanie Turner, et al. v. McDonald's USA, LLC, et al.", metadata={'url': 'https://www.justice.gov/atr/antitrust-case-filings-alpha', 'thumbnail_url': None, 'title': 'Antitrust Division | Antitru

## Instantiate our LLM

In [58]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0.5)

## Tying it all together

In [59]:
from langgraph.prebuilt import chat_agent_executor
from langgraph.checkpoint import MemorySaver

# Create a checkpointer to use memory
memory = MemorySaver()
# the vector store representation of the CSV dataset and the You.com Search tool will both be passed as tools to the agent
tools = [db_retriever_tool, ydc_tool]
agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools, checkpointer=memory)

## Let's try it out!

In [60]:
agent_executor.invoke(input={"messages": "What laws in the US pertain to perjury and is there a recent case in the US that relates to a violation of these laws?"}, config={"configurable": {"thread_id": "xyz_789"}})

{'messages': [HumanMessage(content='What laws in the US pertain to perjury and is there a recent case in the US that relates to a violation of these laws?', id='79701b68-e730-457e-a829-0316d6f01243'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_rpezOI95Dvwx5xw4eRHUMU1d', 'function': {'arguments': '{"query": "perjury laws in the US"}', 'name': 'law_dataset_retriever'}, 'type': 'function'}, {'id': 'call_VzK5aCr8kazBGcuCcmHbdC5X', 'function': {'arguments': '{"query": "recent US case perjury 2023"}', 'name': 'you_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 60, 'prompt_tokens': 128, 'total_tokens': 188}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0d44de67-a4a9-43d8-8ef0-cbda687f1081-0', tool_calls=[{'name': 'law_dataset_retriever', 'args': {'query': 'perjury laws in the US'}, 'id': 'call_rpezOI95Dvwx5xw4eRHUMU1d'}, {'name': 'you_search', 'args

In [61]:
agent_executor.invoke(input={"messages": "What is the most famous US Supreme Court perjury case?"}, config={"configurable": {"thread_id": "xyz_789"}})

{'messages': [HumanMessage(content='What laws in the US pertain to perjury and is there a recent case in the US that relates to a violation of these laws?', id='79701b68-e730-457e-a829-0316d6f01243'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "perjury laws in the US"}', 'name': 'law_dataset_retriever'}, 'id': 'call_rpezOI95Dvwx5xw4eRHUMU1d', 'type': 'function'}, {'function': {'arguments': '{"query": "recent US case perjury 2023"}', 'name': 'you_search'}, 'id': 'call_VzK5aCr8kazBGcuCcmHbdC5X', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'token_usage': {'completion_tokens': 60, 'prompt_tokens': 128, 'total_tokens': 188}}, id='run-0d44de67-a4a9-43d8-8ef0-cbda687f1081-0', tool_calls=[{'name': 'law_dataset_retriever', 'args': {'query': 'perjury laws in the US'}, 'id': 'call_rpezOI95Dvwx5xw4eRHUMU1d'}, {'name': 'you_search', 'args

In [62]:
agent_executor.invoke(input={"messages": "Based on your knowledge of all laws in the US pertaining to perjury, were there any other laws that could have been applied in this case?"}, config={"configurable": {"thread_id": "xyz_789"}})

{'messages': [HumanMessage(content='What laws in the US pertain to perjury and is there a recent case in the US that relates to a violation of these laws?', id='79701b68-e730-457e-a829-0316d6f01243'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "perjury laws in the US"}', 'name': 'law_dataset_retriever'}, 'id': 'call_rpezOI95Dvwx5xw4eRHUMU1d', 'type': 'function'}, {'function': {'arguments': '{"query": "recent US case perjury 2023"}', 'name': 'you_search'}, 'id': 'call_VzK5aCr8kazBGcuCcmHbdC5X', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'token_usage': {'completion_tokens': 60, 'prompt_tokens': 128, 'total_tokens': 188}}, id='run-0d44de67-a4a9-43d8-8ef0-cbda687f1081-0', tool_calls=[{'name': 'law_dataset_retriever', 'args': {'query': 'perjury laws in the US'}, 'id': 'call_rpezOI95Dvwx5xw4eRHUMU1d'}, {'name': 'you_search', 'args

## Let's create a Python class that encapsulates the code above.  This will enable users to easily create chatbots for new use cases with custom CSV datasets

In [63]:
import secrets

class CSV_QA_Bot:
    def __init__(self, llm: ChatOpenAI, csv_files: list[str], num_web_results_to_fetch: int = 10):
        self._llm = llm
        
        docs = self._load_csv_files(csv_files)
        
        # split the docs into chunks, vectorize the chunks and load them into a vector store
        db = self._create_vector_store(docs)
        
        # create a retriever from the vector store
        self._faiss_retriever = db.as_retriever()
        
        # convert this retriever into a Langchain tool
        self._faiss_retriever_tool = create_retriever_tool(
            self._faiss_retriever,
            name = "law_dataset_retriever",
            description = "Retrieve relevant context from the US laws dataset."
        )
        
        # instantiate the YDC search tool in Langchain
        self._ydc_api_wrapper = YouSearchAPIWrapper(num_web_results=num_web_results_to_fetch)
        self._ydc_search_tool = YouSearchTool(api_wrapper=self._ydc_api_wrapper)
        
        
        # create a list of tools that will be supplied to the Langchain agent
        self._tools = [self._faiss_retriever_tool, self._ydc_search_tool]
        
        # Create a checkpointer to use memory
        self._memory = MemorySaver()
        
        # create the agent executor
        self._agent_executor = chat_agent_executor.create_tool_calling_executor(llm, tools, checkpointer=memory)
        
        # generate a thread ID for to keep track of conversation history
        self._thread_id = self._generate_thread_id()

    def _load_csv_files(self, csv_files: list[str]) -> list:
        docs = []
        for file in csv_files:
            data_loader = CSVLoader(file)
            docs.extend(data_loader.load())
        return docs
    
    def _create_vector_store(self, docs: list) -> FAISS:
        text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)
        chunked_docs = text_splitter.split_documents(docs)
        embeddings = OpenAIEmbeddings()
        return FAISS.from_documents(documents=chunked_docs, embedding=embeddings)

    def _generate_thread_id(self) -> str:
        thread_id = secrets.token_urlsafe(16)
        return thread_id
    
    def invoke_bot(self, input_str: str) -> dict:
        input = {"messages": input_str}
        config = {"configurable": {"thread_id": self._thread_id}}
        output = self._agent_executor.invoke(input=input, config=config)
        return output

## Let's try it out!

In [64]:
conversational_agent = CSV_QA_Bot(llm, csv_files=["us_laws_dataset.csv"])

In [65]:
conversational_agent.invoke_bot("What laws in the US pertain to anti-trust and is there a recent case in the USA that pertains to violations of these laws?")

{'messages': [HumanMessage(content='What laws in the US pertain to anti-trust and is there a recent case in the USA that pertains to violations of these laws?', id='cc5221aa-6540-497c-aa62-fe9f5f4530f5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_W0WeWwzeCf4c3TgpBEgXR2Gn', 'function': {'arguments': '{"query": "anti-trust laws in the US"}', 'name': 'law_dataset_retriever'}, 'type': 'function'}, {'id': 'call_Cx0nW4evpzjnJIIxnLDqXDdf', 'function': {'arguments': '{"query": "recent antitrust case in USA 2023"}', 'name': 'you_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 128, 'total_tokens': 190}, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5101fb0f-b9d9-4c75-a5af-32f522392b82-0', tool_calls=[{'name': 'law_dataset_retriever', 'args': {'query': 'anti-trust laws in the US'}, 'id': 'call_W0WeWwzeCf4c3TgpBEgXR2Gn'}, {'name': 'yo

In [66]:
conversational_agent.invoke_bot("What is the most famous US Supreme Court anti-trust case?")

{'messages': [HumanMessage(content='What laws in the US pertain to anti-trust and is there a recent case in the USA that pertains to violations of these laws?', id='cc5221aa-6540-497c-aa62-fe9f5f4530f5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "anti-trust laws in the US"}', 'name': 'law_dataset_retriever'}, 'id': 'call_W0WeWwzeCf4c3TgpBEgXR2Gn', 'type': 'function'}, {'function': {'arguments': '{"query": "recent antitrust case in USA 2023"}', 'name': 'you_search'}, 'id': 'call_Cx0nW4evpzjnJIIxnLDqXDdf', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'token_usage': {'completion_tokens': 62, 'prompt_tokens': 128, 'total_tokens': 190}}, id='run-5101fb0f-b9d9-4c75-a5af-32f522392b82-0', tool_calls=[{'name': 'law_dataset_retriever', 'args': {'query': 'anti-trust laws in the US'}, 'id': 'call_W0WeWwzeCf4c3TgpBEgXR2Gn'}, {'name': 'yo

In [67]:
conversational_agent.invoke_bot("Based on your knowledge of all laws in the USA relating to anti-trust, were there any other laws that could have been applied in this case?")

{'messages': [HumanMessage(content='What laws in the US pertain to anti-trust and is there a recent case in the USA that pertains to violations of these laws?', id='cc5221aa-6540-497c-aa62-fe9f5f4530f5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"query": "anti-trust laws in the US"}', 'name': 'law_dataset_retriever'}, 'id': 'call_W0WeWwzeCf4c3TgpBEgXR2Gn', 'type': 'function'}, {'function': {'arguments': '{"query": "recent antitrust case in USA 2023"}', 'name': 'you_search'}, 'id': 'call_Cx0nW4evpzjnJIIxnLDqXDdf', 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None, 'model_name': 'gpt-4o', 'system_fingerprint': 'fp_319be4768e', 'token_usage': {'completion_tokens': 62, 'prompt_tokens': 128, 'total_tokens': 190}}, id='run-5101fb0f-b9d9-4c75-a5af-32f522392b82-0', tool_calls=[{'name': 'law_dataset_retriever', 'args': {'query': 'anti-trust laws in the US'}, 'id': 'call_W0WeWwzeCf4c3TgpBEgXR2Gn'}, {'name': 'yo